# import

In [32]:
import sys
import os
import os.path as osp
import torch
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np

# constant

# path

In [76]:
data_name = 'player'
data_dir = osp.join('../output', data_name, 'tracking')
save_dir = osp.join('../output', data_name, 'reid')
track_label_path = osp.join('../output/')

In [11]:
track_label_path = '../output/player/person_label.csv'

# function

In [12]:
def listdirs(dir_path, name_condition=None):
    dir_names = os.listdir(dir_path)
    if name_condition is not None:
        dir_names = [dir_name for dir_name in dir_names if dir_name.startswith(name_condition)]
    dir_names.sort()
    return dir_names

In [13]:
def save_pickle(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f)


def load_pickle(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

# data

In [18]:
track_label_df = pd.read_csv(track_label_path)

# main

In [19]:
person_class_id = 1

In [ ]:
sim_th = 0.5

In [83]:
games = listdirs(data_dir, 'game')
for idx, item in track_label_df.iterrows():
    _, tgt_game, tgt_clip, tgt_tracking_id, tgt_name, tgt_loc = item
    game = 'game{}'.format(tgt_game)
    clip = 'Clip{}'.format(tgt_clip)
    tgt_mean_feature_path = osp.join(data_dir, game, clip, '{:05}'.format(tgt_tracking_id), 'mean_feature', 'mean_feature.pkl')
    tgt_mean_f = load_pickle(tgt_mean_feature_path)
    
    print(game)
    game_dir = osp.join(data_dir, game)
    clips = listdirs(game_dir, 'Clip')

    dst_mean_fs = []
    dst_mean_idx = 1
    infos = []
    for clip in tqdm(clips):
        clip_dir = osp.join(game_dir, clip)
        tracking_ids = listdirs(clip_dir)
        for tracking_id in tracking_ids:
            tracking_dir = osp.join(clip_dir, tracking_id)
            mean_feature_path = osp.join(tracking_dir, 'mean_feature', 'mean_feature.pkl')
            if not osp.exists(mean_feature_path):
                print('no mean_feature')
                continue

            dst_mean_f = load_pickle(mean_feature_path)
            sim = np.dot(tgt_mean_f, dst_mean_f)
            info = [game, clip, sim, tracking_id]
            infos.append(info)

    info_df = pd.DataFrame(infos, columns=['game', 'clip', 'similarity', 'tracking_id'])
    info_df = info_df.sort_values(['game', 'clip', 'similarity'], ascending=False)
    info_save_path = osp.join(save_dir, game, '{}.csv'.format(tgt_name))
    os.makedirs(osp.dirname(info_save_path), exist_ok=True)
    info_df.to_csv(info_save_path, index=False)

  0%|          | 0/9 [00:00<?, ?it/s]

game1
game1
game2
game2
game3


  0%|          | 0/15 [00:00<?, ?it/s]

game3
game4
game4
game5


  0%|          | 0/8 [00:00<?, ?it/s]

game5
game6
game6
game7


  0%|          | 0/9 [00:00<?, ?it/s]

game7
game8
game8


100%|██████████| 12/12 [00:00<00:00, 476.05it/s]

game9
game9
game10
game10
